In [ ]:
import selenium

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC


web = 'https://www.amazon.com'

driver = webdriver.Chrome()
driver.get(web)

driver.implicitly_wait(5)
keyword = "wireless charger"
search = driver.find_element(By.ID, 'twotabsearchtextbox')
search.send_keys(keyword)
# click search button
search_button = driver.find_element(By.ID, 'nav-search-submit-button')
search_button.click()

driver.implicitly_wait(5)

product_asin = []
product_name = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []

items = wait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
for item in items:
    # find name
    name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]')
    product_name.append(name.text)

    # find ASIN number 
    data_asin = item.get_attribute("data-asin")
    product_asin.append(data_asin)

    # find price
    whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
    fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')
    
    if whole_price != [] and fraction_price != []:
        price = '.'.join([whole_price[0].text, fraction_price[0].text])
    else:
        price = 0
    product_price.append(price)

    # find ratings box
    ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

    # find ratings and ratings_num
    if ratings_box != []:
        ratings = ratings_box[0].get_attribute('aria-label')
        ratings_num = ratings_box[1].get_attribute('aria-label')
    else:
        ratings, ratings_num = 0, 0
    
    product_ratings.append(ratings)
    product_ratings_num.append(str(ratings_num))
    
    # find link
    try:
        link = item.find_element(By.XPATH, './/a[@class="a-link-normal a-text-normal"]').get_attribute("href")
    except:
        link = ''
    product_link.append(link)

driver.quit()

# to check data scraped
print(product_name)
print(product_asin)
print(product_price)
print(product_ratings)
print(product_ratings_num)
print(product_link)


['KKM Wireless Charger,15W 4 in 1 Fast Charger Station for iPhone 14/14 Max/14 Pro/14 Pro Max/13/12/11/iWatch 1-7/Air-Pods/Samsung S21/S20/S10,Tidy Storage(iWatch Charging Cable Not Included)', 'Wireless Charger, 15W Qi-Certified Max Fast Wireless Charging Pad 2-Pack Compatible with iPhone 13/13 Pro/13 Mini/13 ProMax/12/SE/11, Samsung Galaxy S21/S20/Note 10/Edge Note 20Ultra/S10, AirPods Pro', 'Yootech Wireless Charger,10W Max Fast Wireless Charging Pad Compatible with iPhone 14/14 Plus/14 Pro/14 Pro Max/13/13 Mini/SE 2022/12/11/X/8,Samsung Galaxy S22/S21/S20,AirPods Pro 2(No AC Adapter)', 'Anker 313 Wireless Charger (Stand), Qi-Certified for iPhone 12, 12 Pro Max, SE, 11, 11 Pro, 11 Pro Max, XR, XS Max, 10W Fast-Charging Galaxy S20, S10 (No AC Adapter)', 'Apple MagSafe Charger - Wireless Charger with Fast Charging Capability, Type C Wall Charger, Compatible with iPhone and AirPods', 'Fast Wireless Charger,20W Max Wireless Charging Pad Compatible with iPhone 14/14 Plus/14 Pro/14 Pro Ma

In [15]:
import csv
header = ['product_name','product_asin','product_price','product_ratings', 'product_ratings_num','product_link']
rows = zip(product_name,product_asin,product_price,product_ratings,product_ratings_num,product_link)

filename = "output.csv"
with open(filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)